In [1]:
using InstantiateFromURL
# optionally add arguments to force installation: instantiate = true, precompile = true
github_project("QuantEcon/quantecon-notebooks-julia", version = "0.8.0")

Activated /Users/Shane/Dropbox/GitHub/quantecon-notebooks-julia/Project.toml
Info Project name is quantecon-notebooks-julia, version is 0.8.0


In [2]:
using LinearAlgebra, Statistics
using ForwardDiff, Zygote, Optim, JuMP, Ipopt, BlackBoxOptim, Roots, NLsolve, LeastSquaresOptim
using Optim: converged, maximum, maximizer, minimizer, iterations #some extra functions


┌ Info: Precompiling Zygote [e88e6eb3-aa80-5325-afca-941959d7151f]
└ @ Base loading.jl:1278
┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1278
┌ Info: Precompiling Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]
└ @ Base loading.jl:1278
┌ Info: Precompiling BlackBoxOptim [a134a8b2-14d6-55f6-9291-3336d3ab0209]
└ @ Base loading.jl:1278
┌ Info: Precompiling Roots [f2b01f46-fcfa-551c-844a-d8ac1e96c665]
└ @ Base loading.jl:1278
┌ Info: Precompiling LeastSquaresOptim [0fc2ff8b-aaa3-5acd-a817-1944a5e08891]
└ @ Base loading.jl:1278


In [3]:
using ForwardDiff
h(x) = sin(x[1]) + x[1] * x[2] + sinh(x[1] * x[2]) # multivariate.
x = [1.4 2.2]
@show ForwardDiff.gradient(h,x) # use AD, seeds from x

ForwardDiff.gradient(h, x) = [26.354764961030977 16.663053156992284]


1×2 Array{Float64,2}:
 26.3548  16.6631

In [4]:
#Or, can use complicated functions of many variables
f(x) = sum(sin, x) + prod(tan, x) * sum(sqrt, x)
g = (x) -> ForwardDiff.gradient(f, x); # g() is now the gradient
g(rand(5)) # gradient at a random point
# ForwardDiff.hessian(f,x') # or the hessian

5-element Array{Float64,1}:
 1.25502617687627
 1.249009281667938
 1.71911574674487
 4.823581035646123
 1.4266416174423298

In [5]:
function squareroot(x) #pretending we don't know sqrt()
    z = copy(x) # Initial starting point for Newton’s method
    while abs(z*z - x) > 1e-13
        z = z - (z*z-x)/(2z)
    end
    return z
end
squareroot(2.0)

using ForwardDiff
dsqrt(x) = ForwardDiff.derivative(squareroot, x)
dsqrt(2.0)


0.35355339059327373

In [6]:
using Zygote

h(x, y) = 3x^2 + 2x + 1 + y*x - y
gradient(h, 3.0, 5.0)

(25.0, 2.0)

In [12]:
# create an operator

D(f) = x -> gradient(f,x)[1]

D(sin)(4.0)

-0.6536436208636119

In [19]:
using Statistics
p(x) = mean(abs, x)
#p'([1.0, 3.0, -2.0])
p([1.0, 3.0, -2.0])

2.0

In [22]:
using Optim
using Optim: converged, maximum, maximizer, minimizer, iterations #some extra functions

# minimize
result = optimize(x-> x^2, -2.0, 1.0)

converged(result) || error("Failed to converge in $(iterations(result)) iterations")
xmin = result.minimizer
result.minimum

converged(result) || error("Failed to converge in $(iterations(result)) iterations") = true


0.0

In [23]:
# maximize
f(x) = -x^2
result = maximize(f, -2.0, 1.0)
converged(result) || error("Failed to converge in $(iterations(result)) iterations")
xmin = maximizer(result)
fmax = maximum(result)

-0.0

In [24]:
# unconstrained multivariate

f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x_iv = [0.0, 0.0]
results = optimize(f, x_iv) # i.e. optimize(f, x_iv, NelderMead())


 * Status: success

 * Candidate solution
    Minimizer: [1.00e+00, 1.00e+00]
    Minimum:   3.525527e-09

 * Found with
    Algorithm:     Nelder-Mead
    Initial Point: [0.00e+00, 0.00e+00]

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    60
    f(x) calls:    118


In [26]:
# solve
# min (1-x)^2 + 100(y-x^2)^2)
# st x + y >= 10

using JuMP,Ipopt
m = Model(with_optimizer(Ipopt.Optimizer)) # settings for the solver
@variable(m, x, start = 0.0)
@variable(m, y, start = 0.0)

@NLobjective(m, Min, (1-x)^2 + 100(y-x^2)^2)

JuMP.optimize!(m)
println("x = ", value(x), " y = ", value(y))

# add linear constraint
@constraint(m, x+y == 10)
JuMP.optimize!(m)
println("x = ", value(x), " y = ", value(y))

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [78]:
# Exercise 1
#various rules for differentiation


struct DualNumber{T} <: Real
    val::T
    ϵ::T
end
import Base: +, *, -, ^, /, exp
+(x::DualNumber, y::DualNumber) = DualNumber(x.val + y.val, x.ϵ + y.ϵ)  # dual addition
+(x::DualNumber, a::Number) = DualNumber(x.val + a, x.ϵ)  # i.e. scalar addition, not dual
+(a::Number, x::DualNumber) = DualNumber(x.val + a, x.ϵ)  # i.e. scalar addition, not dual
-(x::DualNumber, y::DualNumber) = DualNumber(x.val - y.val, x.ϵ - y.ϵ)  # dual subtraction
-(x::DualNumber, a::Number) = DualNumber(x.val - a, x.ϵ)  # i.e. scalar subtraction, not dual
-(a::Number, x::DualNumber) = DualNumber(x.val - a, x.ϵ)  # i.e. scalar subtraction, not dual
/(x::DualNumber, y::DualNumber) = DualNumber(x.val/y.val, (y.val*x.ϵ - x.val*y.ϵ)/y.val^2) #dual quotient rule
*(x::DualNumber, y::DualNumber) = DualNumber(x.val*y.val, x.val*y.ϵ + y.val*x.ϵ ) #dual product rule
*(x::DualNumber, a::Number) = DualNumber(a*x.val, a*x.ϵ ) #scalar multiplicaiton
^(a::Number, x::DualNumber) = DualNumber(x.val^a, a*x.val^(a-1)*x.ϵ)



f(x) = x^2.0

x = DualNumber(1.0, 1.0)  # x -> 2.0 + 1.0\epsilon
y = DualNumber(3.0, 1.0)  # i.e. y = 3.0, no derivative


# seeded calculates both the function and the d/dx gradient!
f(x) 


DualNumber{Float64}(1.0, 2.0)

DualNumber{Float64}(1.0, 2.0)